In [43]:
import os
import numpy as np
import cv2

In [44]:
input_folder = 'd:\\15 CT_data\\CPU_test\\'
output_folder = 'd:\\15 CT_data\\CPU_test\\result'
ROI = '400'
method_name = 'otsu'
resize_dims = '0'

In [45]:
img_files = sorted([os.path.join(input_folder, f) for f in os.listdir(input_folder)
                    if f.endswith('.bmp') or f.endswith('.png')])
radius = int(ROI)

In [46]:
def adjust_gamma(image, gamma=1.0):
    # 建立查找表
    inv_gamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** inv_gamma) * 255
                      for i in np.arange(0, 256)]).astype("uint8")

    # 应用伽马校正
    return cv2.LUT(image, table)

In [47]:
test_img = cv2.imread(img_files[0], cv2.IMREAD_GRAYSCALE)
height, width = test_img.shape
test_img = test_img[(height // 2 - radius):(height // 2 + radius), (width // 2 - radius):(width // 2 + radius)]

In [48]:
# gamma_adjusted = adjust_gamma(test_img, gamma=1)
hist = cv2.equalizeHist(test_img)
cv2.namedWindow('test', cv2.WINDOW_NORMAL)
cv2.imshow('test', hist)
cv2.waitKey(0)

32

In [49]:
_, dst = cv2.threshold(gamma_adjusted, 0, 255, cv2.THRESH_OTSU)
pore = np.sum(dst) / 255 / dst.shape[0] / dst.shape[1]
print(pore)
cv2.namedWindow('threshold', cv2.WINDOW_NORMAL)
cv2.imshow('threshold', dst)
cv2.waitKey(0)
cv2.destroyAllWindows()

0.5466546875
